# Cats vs Dogs Transfer Learning Notebook
Comprehensive end-to-end workflow: dataset prep, transfer learning (feature extraction + fine-tuning), evaluation, error analysis, export, and optional tuning / tracking.

Run sections sequentially; cells are modular. Adjust `DATA_ROOT` to where you extracted the Kaggle dataset.


In [ ]:
# 1. Project Configuration & Environment Check
import os, sys, random, platform, multiprocessing, math, time, pathlib
import numpy as np
import tensorflow as tf
print('Python version:', sys.version)
print('TensorFlow version:', tf.__version__)
print('Platform:', platform.platform())
print('Physical cores:', multiprocessing.cpu_count())
print('GPUs:', len(tf.config.list_physical_devices('GPU')))
SEED = 42
random.seed(SEED); np.random.seed(SEED); tf.random.set_seed(SEED)
# Mixed precision if GPU with float16 capability
from tensorflow.keras import mixed_precision
if tf.config.list_physical_devices('GPU'):
    try:
        mixed_precision.set_global_policy('mixed_float16')
        print('Mixed precision enabled')
    except Exception as e:
        print('Could not enable mixed precision:', e)
DATA_ROOT = pathlib.Path('../data/PetImages')  # adjust if needed
RAW_ARCHIVE = pathlib.Path('../data/raw')
SPLIT_ROOT = pathlib.Path('../data/split')
for p in [DATA_ROOT, RAW_ARCHIVE, SPLIT_ROOT]:
    p.mkdir(parents=True, exist_ok=True)


In [ ]:
# 2. Install & Import Dependencies
# (If running in a fresh environment you may need to pip install; skipped here assuming requirements installed.)
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, precision_recall_fscore_support
import itertools


In [ ]:
# 3. Download & Prepare Dataset (Kaggle Cats vs Dogs)
# Placeholder: Kaggle download requires kaggle.json credentials.
# Example (uncomment if credentials configured):
# !kaggle datasets download -d tongpython/cat-and-dog -p ../data/raw
# !unzip -o ../data/raw/cat-and-dog.zip -d ../data/raw
print('Assuming dataset already present at', DATA_ROOT)


In [ ]:
# 4. Directory Structure Verification
from collections import Counter
classes = []
counts = {}
if DATA_ROOT.exists():
    for cls in sorted([d for d in DATA_ROOT.iterdir() if d.is_dir()]):
        classes.append(cls.name)
        counts[cls.name] = sum(1 for _ in cls.glob('*.jpg'))
print('Classes:', classes)
print('Counts:', counts)
assert all(c > 0 for c in counts.values()), 'One of the classes has zero images.'
